# Library Calling :

**In this Paper I will be predecting the range of the IMDb Score rating of movies according to the features in the data **
I achieved a 0.83 accuracy you can follow along the code


In [ ]:
#!pip install sidetable
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
#import sidetable
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder


# Data Preparation

## Data Cleaning

In [ ]:
data=pd.read_csv("../input/movies-on-netflix-prime-video-hulu-and-disney/MoviesOnStreamingPlatforms_updated.csv")

In [ ]:
data.drop(columns="Unnamed: 0",inplace=True)


In [ ]:
data.head()

In [ ]:
data.info()

**Seems like there is a lot of missing values let's see the percentages :**

In [ ]:
data.isna().mean()*100

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(data)

* Rotten Tomatoes and Age column lack a lot of values so let's drop them.
* We will replace the rating IMDb by the mean.
* The other missing values are with low percentages so let's drop them



In [ ]:
data.drop(["Rotten Tomatoes","Age","Type"],axis=1,inplace=True)
data.dropna(axis=0,inplace=True,how="any")
data["ID"]=[i for i in range(len(data))]
data.set_index("ID",inplace=True)

In [ ]:
data

In [ ]:
data["IMDb"] = [float(score[0:3]) for score in data["IMDb"]]

# let's see some visualisation:

## Let's Discover the Data:

Let's see the genres that we have :

In [ ]:
genres=[]
for s in data.Genres.dropna():
  l=s.split(sep=",")
  for s2 in l:
    if not( s2 in genres ):
      genres.append(s2)
print(genres)
print(len(genres))

Let's see the countries that we have :

In [ ]:
country=[]
for s in data.Country.dropna():
  l=s.split(sep=",")
  for s2 in l:
    if not( s2 in country ):
      country.append(s2)
print(country)
print(len(country))

Let's see the languages that we have :

In [ ]:
language=[]
for s in data.Language.dropna():
  l=s.split(sep=",")
  for s2 in l:
    if not( s2 in language ):
      language.append(s2)
print(language)
print(len(language))

## Some viz:

let's see the distribution of the IMDb score

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Distribution of IMDb Ratings", fontsize=15)
sns.distplot(data['IMDb'])

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Movie Length Distribution", fontsize=15)
sns.distplot(data.Runtime)

Soo the majority of the scores are between 5 and 8. 
so a question comes here : which of the directors is the best rated? 

In [ ]:
data[['IMDb','Directors']].groupby('Directors').agg(['mean','count']).sort_values(by=('IMDb','mean'), ascending=False)

Let's see those with a better count of films:

In [ ]:
directors = data[['IMDb','Directors']].groupby('Directors').agg(['mean','count'])
directors[directors[('IMDb', 'count')] > 5].sort_values(by=('IMDb','mean'), ascending=False).head(10)

let's see now the platforms:
which platform host the best films ?

In [ ]:
streams = ['Netflix', 'Hulu', 'Prime Video', 'Disney+']
dict_a = {}
for stream in streams:
    a = data[['IMDb', stream]].groupby([stream]).agg(['mean',
        'count']).iloc[1,:].values
    dict_a[stream] = a
    df_stream = pd.DataFrame(dict_a, 
                index=['Avg_IMDb','Number of Moviews'])
df_stream

What about the corrolation: 

In [ ]:
data.corr()

In [ ]:
sns.heatmap(data.corr(),
            xticklabels=data.columns,
            yticklabels=data.columns
           )

seems like there isn't any obvious interesting corrolation for the moment 

In [ ]:
df=data
df["Score"]=2
for i,n in enumerate(df.IMDb):
  if 0<=n<3.0:
    df.loc[i,"Score"]=0
  elif 3.0<=n<5.0:
    df.loc[i,"Score"]=1
  elif 5.0<=n<8.0:
    df.loc[i,"Score"]=2
  elif 8.0<=n<9.0:
    df.loc[i,"Score"]=3
  elif 9.0<=n<10.0:
    df.loc[i,"Score"]=4

In [ ]:
df.isna().sum()

In [ ]:
genres=[]
for s in data.Genres:
  s=str(s)
  l=s.split(sep=",")
  for s2 in l:
    if not( s2 in genres ):
      genres.append(s2)
      data[s2]=0
print(genres)
print(len(genres))
lt=[]
for i,s in enumerate(data.Genres):
  s=str(s)
  if s!= "nan" :
    l=s.split(sep=",")
    for s2 in l:
      data.loc[i,s2]=1

In [ ]:
data


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder =LabelEncoder()
#dirators and name
data["Directors"]=encoder.fit_transform(data["Directors"])
data["Title"]=encoder.fit_transform(data["Title"])
data["Country"]=encoder.fit_transform(data["Country"])
data["Language"]=encoder.fit_transform(data["Language"])
data["Genres"]=encoder.fit_transform(data["Genres"])

In [ ]:
df.columns

In [ ]:
x=df[['Title', 'Year', 'Netflix', 'Hulu', 'Prime Video', 'Disney+',
       'Directors', 'Genres', 'Country', 'Language', 'Runtime','Action', 'Adventure', 'Sci-Fi', 'Thriller', 'Comedy', 'Western',
       'Animation', 'Family', 'Biography', 'Drama', 'Music', 'War', 'Crime',
       'Fantasy', 'Romance', 'History', 'Mystery', 'Horror', 'Sport',
       'Documentary', 'Musical', 'News', 'Short', 'Reality-TV', 'Film-Noir',
       'Talk-Show']]

y=df['Score']



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=30) #split our data with test size of 20% 

knn=KNeighborsClassifier(n_neighbors=22) #build our knn classifier
knn.fit(x_train,y_train) #Training KNN classifier
y_pred=knn.predict(x_test)  #Testing
print('Acuuracy=',accuracy_score(y_pred,y_test))


In [ ]:
n_neighbors=30
scores=[]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=30)
for k in range(1,30):
    knn=KNeighborsClassifier(n_neighbors-k)
    knn.fit(x_train,y_train)
    y_pred=knn.predict(x_test)
    print('Acuuracy=',accuracy_score(y_pred,y_test))


So the Knn algorithme gave us an intresting result. 
Let's try the random forest classifier but the addition of the other features may overfit...

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import tree   
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.20,random_state=10)

#applying tree algorithm
tree1 = tree.DecisionTreeClassifier()  
tree1.fit(x_train, y_train)   #fitting our model
y_pred=tree1.predict(x_test)   # evaluating our model
print("score:{}".format(accuracy_score(y_test, y_pred)))

In [ ]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier #Importing Random Forest Classifier
from sklearn import metrics  # Importing metrics to test accuracy

In [ ]:
max=0 
for i in range(1,40):
  clf=RandomForestClassifier(n_estimators=i)  #Creating a random forest with 100 decision trees
  clf.fit(x_train, y_train)  #Training our model
  y_pred=clf.predict(x_test) #testing our model
  if ( max < metrics.accuracy_score(y_test, y_pred)):
    max=metrics.accuracy_score(y_test, y_pred)
  print("Accuracy:", metrics.accuracy_score(y_test, y_pred)," i= ",i)  #Measuring the accuracy of our model

In [ ]:
print(max)

In [ ]:
data.info()

So Finally we can say that the best algorithm is the Random forest classifier also the knn neighboors worked fine withe the same accuracy of 83 % .